In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import seaborn as sns
import eli5
from IPython.display import Image

In [29]:
for dirname, _, filenames in os.walk('../bbc_news_Data/bbc-fulltext (document classification)/bbc/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../bbc_news_Data/bbc-fulltext (document classification)/bbc/README.TXT
../bbc_news_Data/bbc-fulltext (document classification)/bbc/business/001.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/business/002.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/business/003.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/business/004.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/business/005.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/business/006.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/business/007.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/business/008.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/business/009.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/business/010.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/business/011.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/business/012.txt
../bb

../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/299.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/300.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/301.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/302.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/303.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/304.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/305.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/306.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/307.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/308.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/309.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/310.txt
../bbc_news_Data/bbc-fulltext (document classification)/bbc/sport/311.txt
../bbc_news_Data/bbc-fulltext (documen

In [ ]:
'''
1. Walk through each folder.
2. Pick the complete path, directory and filename.
3. Read the file into a string line by line
4. Pick the first line as Title
5. And the remaining concat to form the text.
6. Set the directory name as label.
'''

In [30]:
# Step 1 - Get the file details
directory = []
file = []
title = []
text = []
label = []
datapath = '../bbc_news_Data/bbc-fulltext (document classification)/bbc/' 
for dirname, _ , filenames in os.walk(datapath):
    #print('Directory: ', dirname)
    #print('Subdir: ', dirname.split('/')[-1])
    # remove the Readme.txt file
    # will not find file in the second iteration so we skip the error
    try:
        filenames.remove('.DS_Store')
        filenames.remove('README.TXT')
    except:
        pass
    for filename in filenames:
        directory.append(dirname)
        file.append(filename)
        label.append(dirname.split('/')[-1])
        #print(filename)
        fullpathfile = os.path.join(dirname,filename)
        with open(fullpathfile, 'r', encoding="utf8", errors='ignore') as infile:
            intext = ''
            firstline = True
            for line in infile:
                if firstline:
                    title.append(line.replace('\n',''))
                    firstline = False
                else:
                    intext = intext + ' ' + line.replace('\n','')
            text.append(intext)


In [31]:
## Df
fulldf = pd.DataFrame(list(zip(directory, file, title, text, label)), 
               columns =['directory', 'file', 'title', 'text', 'label'])

df = fulldf.filter(['title','text','label'], axis=1)
df['text'] = df['title'] + df['text']
df = df.drop(columns=["title"])
df = df[["label", "text"]]
print("FullDf : ", fulldf.shape)
print("DF : ", df.shape)


FullDf :  (2226, 5)
DF :  (2226, 2)


In [32]:
fulldf.head()
#df.head()

directory        file  \
0  ../bbc_news_Data/bbc-fulltext (document classi...  README.TXT   
1  ../bbc_news_Data/bbc-fulltext (document classi...     001.txt   
2  ../bbc_news_Data/bbc-fulltext (document classi...     002.txt   
3  ../bbc_news_Data/bbc-fulltext (document classi...     003.txt   
4  ../bbc_news_Data/bbc-fulltext (document classi...     004.txt   

                                               title  \
0  Consists of 2225 documents from the BBC news w...   
1                  Ad sales boost Time Warner profit   
2                   Dollar gains on Greenspan speech   
3                  Yukos unit buyer faces loan claim   
4                  High fuel prices hit BA's profits   

                                                text     label  
0   Natural Classes: 5 (business, entertainment, ...            
1    Quarterly profits at US media giant TimeWarn...  business  
2    The dollar has hit its highest level against...  business  
3    The owners of embattled Russian oil giant Yu...  business  
4    British Airways has blamed high fuel prices ...  business

In [33]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3)
dev, test = train_test_split(test, test_size=0.5)
print("Train DF: ",train.shape)
print("Dev DF: ",dev.shape)
print("Test DF: ",test.shape)



Train DF:  (1558, 2)
Dev DF:  (334, 2)
Test DF:  (334, 2)


In [34]:

import os
os.makedirs('classify', exist_ok=True)
os.makedirs('classify/model', exist_ok=True)
os.makedirs('classify/data', exist_ok=True)
train.to_csv("./classify/data/train.csv")
dev.to_csv("./classify/data/dev.csv")
test.to_csv("./classify/data/test.csv")



In [35]:

from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

# this is the folder in which train, test and dev files reside
data_folder = 'classify/data'

# column format indicating which columns hold the text and label(s)
column_name_map = {2: "text", 1: "label"}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True,
                                         delimiter=',',    # tab-separated files
) 
label_dict = corpus.make_label_dictionary()



2020-09-17 09:10:04,576 Reading data from classify/data
2020-09-17 09:10:04,577 Train: classify/data/train.csv
2020-09-17 09:10:04,578 Dev: classify/data/dev.csv
2020-09-17 09:10:04,579 Test: classify/data/test.csv
2020-09-17 09:10:04,668 Computing label dictionary. Progress:


100%|██████████| 1891/1891 [00:13<00:00, 142.38it/s]

2020-09-17 09:10:18,101 [b'sport', b'politics', b'business', b'entertainment', b'tech']


In [26]:
label_dict

In [36]:

from flair.embeddings import TransformerDocumentEmbeddings
document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)
# network problem not work in office 

KeyboardInterrupt: 